In [1]:
from math import floor
import itertools

def GetGoppaPolynomial(polynomial_ring, polynomial_degree):
            while 1:
                irr_poly = polynomial_ring.random_element(polynomial_degree)
                irr_poly_list = irr_poly.list()
                irr_poly_list[-1] = 1
                irr_poly = polynomial_ring(irr_poly_list)
                if irr_poly.degree() != polynomial_degree:
                    continue
                elif irr_poly.is_irreducible():
                    break
                else:
                    continue

            return irr_poly

class Niederreiter:
        def __init__(self):
            m = 4
            n = 2 ** m
            t = 2
            F_2m = GF(n, 'Z', modulus='random')
            PR_F_2m = PolynomialRing(F_2m, 'X')
            Z = F_2m.gen()
            X = PR_F_2m.gen()
            irr_poly = GetGoppaPolynomial(PR_F_2m, t)

            goppa_code = GoppaCode(n, m, irr_poly)

            k = goppa_code.generator_matrix().nrows()

            # Random binary non singulary matrix -> S
            S = matrix(GF(2), n - k, [random() < 0.5 for _ in range((n - k) ^ 2)])
            while rank(S) < n - k:
                S[floor((n - k) * random()), floor((n - k) * random())] += 1
                
            # parity check matrix for code -> H
            H = goppa_code.parity_check_matrix()
            
            # Random permutation matrix -> P
            rng = range(n)
            P = matrix(GF(2), n)
            for i in range(n):
                p = floor(len(rng) * random())
                P[i, rng[p]] = 1
                rng = [*rng[:p], *rng[p + 1:]]

            self._m_GoppaCode = goppa_code
            self._g = irr_poly
            self._t = self._g.degree()
            self._S = S
            self.H = H
            self._P = P
            self._PublicKey = S * H * P   

        def encrypt(self, message):
            # verify length of message
            assert (message.ncols() == self._PublicKey.ncols()), "Message is not of the correct length"             
            code_word = self._PublicKey*(message.transpose())
            return code_word.transpose()

        def decrypt(self, received_word):
            # verify length of received word
            received_word = received_word.transpose()
            assert (received_word.nrows() == self._PublicKey.nrows()), "Received word is not of the correct row length"
            assert (received_word.ncols() == 1), "Received word is not of the correct column length"
            
            message = ~(self._S)*received_word
            
            # Syndrome decoding
            t = self._t
            m = message.nrows()/t
            g = self._m_GoppaCode.goppa_polynomial()
            F2 = GF(2)
            F_2m = g.base_ring() 
            Z = F_2m.gen()
            PR_F_2m = g.parent()
            X = PR_F_2m.gen()
            syndrome_poly = 0
                
            for i in range(t):
                tmp = []
                for j in range(m):
                    tmp.append(message[i*m+j,0])
                syndrome_poly += F_2m(tmp[::1])*X^i
               
            message = self._m_GoppaCode.SyndromeDecode(syndrome_poly)
            
            message = message*self._P
            return message